In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip drive/MyDrive/MURA-v1.1.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  drive/MyDrive/MURA-v1.1.zip
replace MURA-v1.1/train_labeled_studies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace MURA-v1.1/valid_labeled_studies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
N
N


In [ ]:
!pip install wandb
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports

In [ ]:
import gc
import re
import os

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pandas as pd
pd.set_option('display.max_columns', None)
import cv2

import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Conv2D, Dense, Dropout, Flatten, MaxPool2D, BatchNormalization, GlobalAveragePooling2D # Layers to be used for building our model
from tensorflow.keras.models import Model # The class used to create a model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import backend as K
from tensorflow.random import set_seed
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import concatenate
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

from typing import List, Dict, Optional, Tuple, Any, Union

import wandb
from wandb.keras import WandbCallback

In [ ]:
# NEW on TPU in TensorFlow 24: shorter cross-compatible TPU/GPU/multi-GPU/cluster-GPU detection code

try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

Num GPUs Available:  1
Number of accelerators:  1


# Globals

In [ ]:
# define path under which MURA-v1.1/ is located:
print(os.getcwd())
root_path: str = os.getcwd()

/content


# Weights & Biases

In order to make experiment tracking easier we will use [Weights & Biases](wandb.ai/home), which offers a free lisence for academic purposes. For the sake of this assignment a team has been created:https://wandb.ai/aueb. Access can be granted by contacting the authors.

* Note that this code assumes that you have already set up a Wandb account and API key. If you haven't done so yet, you will need to sign up for a free account at https://wandb.ai/ and follow the instructions there to obtain your API key.



In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Load Data

Data has the following structure:

```
├──MURA
  ├──train_image_paths.csv
  ├──train_labeled_studies.csv
  ├──valid_image_paths.csv
  ├──valid_labeled_studies.csv
  ├──train
  │   └─ BODY PART
  │       └─ patientxxx
  │          .
  │          .
  │          .
  │    .
  │    .
  │    .
  └──test
      └─ BODY PART
          └─ patientxxx
             .
             .
             .
       .
       .
       .  
```

We will create a dataframe that uses these paths as rows and extract any information needed.

In [ ]:
def extract_set_category(string: str) -> dict:
    """
    Extracts the 'set_type' and 'category' from a given string using regular expressions.

    Parameters:
        string (str): A string containing the 'set' and 'category' information.

    Returns:
        dict: A dictionary containing the 'set' and 'category' information.
    """

    pattern = r".*(?P<set_type>train|valid)/(?P<category>XR_[A-Z]+)/(?P<patient_id>patient\d+)/study.*"
    match = re.match(pattern, string)
    if match:
        return {'set_type': match.group('set_type'), 'category': match.group('category'), 'patient_id': match.group('patient_id')}
    else:
        return None
    
def generate_path_df(dataset_type: str, dataset_path: str = root_path) -> pd.DataFrame:
    """
    Reads in the image paths and labels for a given dataset type (train or valid) from the MURA dataset.
    Returns a pandas DataFrame containing the image paths, labels, and the dataset type.
    
    Parameters:
        dataset_type (str): The type of dataset to read in (train or valid)
        dataset_path (str): The path to the MURA dataset folder (default: '/content')
    
    Returns:
        pd.DataFrame: A pandas DataFrame containing the image paths, labels, and dataset type.
    """
        
    # Read in the image paths csv file and assign the column name 'image_path'
    train_label_paths = pd.read_csv(f"{dataset_path}/MURA-v1.1/{dataset_type}_image_paths.csv", header=None, names=['image_path'])
    
    # Extract the path to the folder containing the image file and create a new column 'path'
    train_label_paths["path"] = train_label_paths.apply(lambda x: "/".join(x['image_path'].split("/")[:-1]) + "/", axis=1)
    
    # Read in the labeled studies csv file and assign column names 'path' and 'label'
    train_labels = pd.read_csv(f"{dataset_path}/MURA-v1.1/{dataset_type}_labeled_studies.csv", header=None, names=['path', 'label'])
    
    # Merge the two DataFrames on the 'path' column and create a new column 'image_type'
    _df = train_labels.merge(train_label_paths, on='path', how='left')
    
    # Check that the length of the two DataFrames match
    assert len(train_label_paths) == len(_df)
    
    return _df

def generate_dataframes(dataset_path: str = root_path) -> pd.DataFrame:
    """Perfoms actions needed to load the dataset with image paths and additional info"""
    
    # read train test_dataframe
    train: pd.DataFrame = generate_path_df(dataset_type="train")
    test: pd.DataFrame = generate_path_df(dataset_type="valid")

    # join dataframes
    _df = pd.concat([train, test]).reset_index()

    # Apply the extract_set_category function to each row of the DataFrame.
    _df = pd.concat([_df, pd.DataFrame(_df['path'].apply(lambda x: extract_set_category(x)).tolist())], axis=1)
    mapping: dict = {1: "abnormal", 0: "normal"}
    _df['label_type'] = _df['label'].apply(lambda x: mapping[x])
    
    
    # re-order columns
    cols = list(_df.columns)
    cols.remove("label")
    cols.append("label")
    _df = _df[cols]
    
    _df.drop(["index"], axis=1, inplace=True)
    
    return _df

In [ ]:
data = generate_dataframes()
data.head()

,path,image_path,set_type,category,patient_id,label_type,label
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,train,XR_SHOULDER,patient00001,abnormal,1
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,train,XR_SHOULDER,patient00001,abnormal,1
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,train,XR_SHOULDER,patient00001,abnormal,1
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,train,XR_SHOULDER,patient00002,abnormal,1
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,train,XR_SHOULDER,patient00002,abnormal,1


# Train Dev Test (valid) split

* in order not to have any dependencies on the order of the data, we will shuffle the data. Moreover, 10% of the input data will be used as validation and 10% as test.

As described in the Paper we will make sure not to have overlap between patients in the various sets. We shuffle the patients, and then split the dataset  

In [ ]:
# take unique patient_ids and shuffle
patients = data.patient_id.unique()
np.random.seed(41)
np.random.shuffle(patients)

# split three list of patient_ids
length_80 = patients[:int(len(patients)*0.83)]
length_80_90 = patients[int(len(patients)*0.83):int(len(patients)*0.90)]
length_90_100 = patients[int(len(patients)*0.90):]

# sanity check
data.loc[data.patient_id.isin(length_80)].describe()\
.join(data.loc[data.patient_id.isin(length_80_90)].describe(),rsuffix="_Validation")\
.join(data.loc[data.patient_id.isin(length_90_100)].describe(), rsuffix="_Test")

,label,label_Validation,label_Test
count,33209.000000,2815.000000,3981.000000
mean,0.411334,0.395027,0.409696
std,0.492083,0.488943,0.491839
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000


* Update the labels for each set & save dataframes to variables

In [ ]:
# update labels
data.loc[data.patient_id.isin(length_80), "set_type"] = "train"
data.loc[data.patient_id.isin(length_80_90), "set_type"] = "validation"
data.loc[data.patient_id.isin(length_90_100), "set_type"] = "test"

# Convert to sstring
data["label"] = data["label"].astype(str)

train: pd.DataFrame = data.loc[data.set_type == 'train']
valid: pd.DataFrame = data.loc[data.set_type == 'validation']
test: pd.DataFrame = data.loc[data.set_type == 'test']
    
train.head()

,path,image_path,set_type,category,patient_id,label_type,label
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,train,XR_SHOULDER,patient00002,abnormal,1
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,train,XR_SHOULDER,patient00002,abnormal,1
5,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,train,XR_SHOULDER,patient00002,abnormal,1
6,MURA-v1.1/train/XR_SHOULDER/patient00003/study...,MURA-v1.1/train/XR_SHOULDER/patient00003/study...,train,XR_SHOULDER,patient00003,abnormal,1
7,MURA-v1.1/train/XR_SHOULDER/patient00003/study...,MURA-v1.1/train/XR_SHOULDER/patient00003/study...,train,XR_SHOULDER,patient00003,abnormal,1


* visualize new class distribution. It seems that the selected seed provides a balanced set to train.

# Data Augmentation

* As mentioned in the introductions, some pre-processing steps will take place, similar to the ones used in the paper.

In [ ]:
from PIL import ImageEnhance, Image
# Define the preprocessing function
def normalize(x):
    x /= 255.0  # Scale pixel values to [0, 1]
    
    # normalize to imagenet mean and std
    x -= [0.485, 0.456, 0.406]    
    x /= [0.229, 0.224, 0.225]     
    
    return x

def augment_image(image):
    image = Image.fromarray(np.uint8(image))
    image = ImageEnhance.Brightness(image).enhance(np.random.uniform(0.8, 1.2))
    image = ImageEnhance.Contrast(image).enhance(np.random.uniform(0.8, 1.2))
    image = np.array(image)/255.0  # normalize
    return image

In [ ]:
x_col='image_path'
y_col='label'
batch_size=128
seed=42
shuffle=True
class_mode='binary'
target_size=(224,224)

# create Data generators
train_datagen = ImageDataGenerator(
    rotation_range=30,
    horizontal_flip=True,
    preprocessing_function=augment_image
)


valid_test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)


# prepare iterators
train_iterator = train_datagen.flow_from_dataframe(
    dataframe=train,
    x_col=x_col,
    y_col=y_col,
    batch_size=batch_size,
    seed=seed,
    shuffle=shuffle,
    class_mode='binary',
    target_size=target_size
)

valid_iterator = valid_test_datagen.flow_from_dataframe(
    dataframe=valid,
    x_col=x_col,
    y_col=y_col,
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode='binary',
    target_size=target_size
)

test_iterator = valid_test_datagen.flow_from_dataframe(
    dataframe=test,
    x_col=x_col,
    y_col=y_col,
    batch_size=batch_size,
    seed=42,
    shuffle=False,
    class_mode='binary',
    target_size=target_size
)

Found 33209 validated image filenames belonging to 2 classes.
Found 2815 validated image filenames belonging to 2 classes.
Found 3981 validated image filenames belonging to 2 classes.


In [ ]:
import tensorflow_addons as tfa
def opt_es(learning_rate=0.0001, monitor='val_loss', patience=10) -> tuple:
    """return the Adam optimizer and the readly stopping"""
    optimizer = Adam(learning_rate=learning_rate)
    early_stopping = EarlyStopping(
            monitor=monitor,
            patience=patience,
            verbose=1,
            restore_best_weights=True
        )
    reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
    return optimizer, early_stopping, reduceLR

def clean_up(model):
    K.clear_session()
    del model
    gc.collect()

def print_eval(hs, _eval) -> None:
    """Prints Train, validation and test metrics for an input hs object"""

    print("Train Loss     : {0:.5f}".format(hs.history['loss'][-1]))
    print("Validation Loss: {0:.5f}".format(hs.history['val_loss'][-1]))
    print("Test Loss      : {0:.5f}".format(_eval[0]))
    print("---")
    print("Train Accuracy     : {0:.5f}".format(hs.history['accuracy'][-1]))
    print("Validation Accuracy: {0:.5f}".format(hs.history['val_accuracy'][-1]))
    print("Test Accuracy      : {0:.5f}".format(_eval[1]))

In [ ]:
def train_model_stacked(
        train_iterator,
        valid_iterator,
        optimizer: tf.keras.optimizers,
        metrics: list,
        callbacks: Optional[List[Any]] = None, 
        verbose: int = 0,
        epochs: int = 20,
        input_shape: tuple = (224, 224, 3),
        train: bool = True) -> tuple:
    
    
    np.random.seed(42) # Define the seed for numpy to have reproducible experiments.
    set_seed(42) # Define the seed for Tensorflow to have reproducible experiments.
    
    mobilenet_base = MobileNetV2(weights = 'imagenet',
                                 input_shape = input_shape,
                                 include_top = False)

    densenet_base = DenseNet169(weights = 'imagenet', 
                                input_shape = input_shape,
                                include_top = False)
    
    for layer in mobilenet_base.layers:
        layer.trainable =  False
    for layer in densenet_base.layers:
        layer.trainable = False

    # Define the input layer.
    _input = Input(
        shape=input_shape,
        name='Input'
    )
    model_mobilenet = mobilenet_base(_input)
    model_mobilenet = GlobalAveragePooling2D(
        name="Pooling2D-Mobilenet"
        )(model_mobilenet)
    output_mobilenet = Flatten(
        name='Flatten-Mobilenet'
        )(model_mobilenet)

    model_densenet = densenet_base(_input)
    model_densenet = GlobalAveragePooling2D(
        name="Pooling2D-Densenet"
        )(model_densenet)
    output_densenet = Flatten(
        name='Flatten-Densenet'
        )(model_densenet)

    merged = tf.keras.layers.Concatenate(
        name='Concat'
        )([output_mobilenet, output_densenet])

    x = BatchNormalization(
        name='BatchNormalization-Merged'
        )(merged)
    x = Dense(units=256,
              activation = 'relu'
              )(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization(
        name='BatchNormalization-out1'
        )(x)
    x = Dense(
        units=128,
        activation = 'relu'
        )(x)
    x = Dropout(0.5)(x)
    output = Dense(
        units=1, 
        activation = 'sigmoid'
        )(x)

    # Define the model and train it.
    model = Model(inputs=_input, outputs=output)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)
    model.summary() # Print a description of the model.
    if train:
        hs = model.fit(
            train_iterator,
            validation_data=valid_iterator,
            epochs=epochs,
            verbose=verbose,
            callbacks=callbacks,
            shuffle=True
        )
        print('Finished training.')
        print('------------------')
        
        return model, hs
    else:
        return model, None

In [ ]:
# init wandb
wandb.init(project="Deep_Learning_2", 
           name="Stacked_frozen")


# Metrics and optimizer
metrics = [tfa.metrics.CohenKappa(name="cohen_kappa", num_classes=2),
           'accuracy']
optimizer, early_stopping, reduceLR = opt_es(0.0001)

model, hs = train_model_stacked(train_iterator=train_iterator,
    valid_iterator=valid_iterator,
    optimizer=optimizer,
    epochs=20,
    callbacks=[WandbCallback(), early_stopping, reduceLR],
    metrics=metrics,
    verbose=1)


# evaluate
# Evaluate on test data and show all the results.
_eval = model.evaluate(test_iterator, verbose=1)
print_eval(hs, _eval)
clean_up(model=model)
wandb.finish()

accuracy,▁
cohen_kappa,▁
epoch,▁
loss,▁
val_accuracy,▁
val_cohen_kappa,▁
val_loss,▁
accuracy,0.69773
best_epoch,0
best_val_loss,0.48698
cohen_kappa,0.36204


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 mobilenetv2_1.00_224 (Function  (None, 7, 7, 1280)  2257984     ['Input[0][0]']                  
 al)                                                                                              
                                                                                                  
 densenet169 (Functional)       (None, 7, 7, 1664)   12642880    ['Input[0][0]']                  
                                                                                              

wandb: Adding directory to artifact (/content/wandb/run-20230325_180431-wz51cumy/files/model-best)... Done. 0.3s


260/260 [==============================] - 613s 2s/step - loss: 0.7611 - cohen_kappa: 0.1649 - accuracy: 0.5909 - val_loss: 0.5491 - val_cohen_kappa: 0.4378 - val_accuracy: 0.7382 - lr: 1.0000e-04
Epoch 2/20
260/260 [==============================] - ETA: 0s - loss: 0.6731 - cohen_kappa: 0.2680 - accuracy: 0.6496

wandb: Adding directory to artifact (/content/wandb/run-20230325_180431-wz51cumy/files/model-best)... Done. 0.2s


...

wandb: Adding directory to artifact (/content/wandb/run-20230325_180431-wz51cumy/files/model-best)... Done. 0.2s


260/260 [==============================] - 598s 2s/step - loss: 0.5189 - cohen_kappa: 0.4566 - accuracy: 0.7447 - val_loss: 0.4767 - val_cohen_kappa: 0.5120 - val_accuracy: 0.7773 - lr: 1.0000e-04
Epoch 17/20
260/260 [==============================] - ETA: 0s - loss: 0.5148 - cohen_kappa: 0.4684 - accuracy: 0.7500

wandb: Adding directory to artifact (/content/wandb/run-20230325_180431-wz51cumy/files/model-best)... Done. 0.2s


260/260 [==============================] - 601s 2s/step - loss: 0.5148 - cohen_kappa: 0.4684 - accuracy: 0.7500 - val_loss: 0.4737 - val_cohen_kappa: 0.5135 - val_accuracy: 0.7787 - lr: 1.0000e-04
Epoch 18/20
260/260 [==============================] - 512s 2s/step - loss: 0.5092 - cohen_kappa: 0.4741 - accuracy: 0.7529 - val_loss: 0.4749 - val_cohen_kappa: 0.5064 - val_accuracy: 0.7755 - lr: 1.0000e-04
Epoch 19/20
260/260 [==============================] - 512s 2s/step - loss: 0.5112 - cohen_kappa: 0.4786 - accuracy: 0.7549 - val_loss: 0.4744 - val_cohen_kappa: 0.5173 - val_accuracy: 0.7801 - lr: 1.0000e-04
Epoch 20/20
260/260 [==============================] - 512s 2s/step - loss: 0.5082 - cohen_kappa: 0.4774 - accuracy: 0.7545 - val_loss: 0.4741 - val_cohen_kappa: 0.5103 - val_accuracy: 0.7769 - lr: 1.0000e-04
Finished training.
------------------
32/32 [==============================] - 22s 687ms/step - loss: 0.5093 - cohen_kappa: 0.5034 - accuracy: 0.7704
Train Loss     : 0.50820
V

accuracy,▁▄▄▅▅▆▆▆▇▇▇▇▇▇▇█████
cohen_kappa,▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇█████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▆▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
val_accuracy,▁▃▅▅▅▆▇▆▇▇▇▇█▇▇██▇█▇
val_cohen_kappa,▁▃▄▅▅▆▇▆▇▇▇▇█▇▇██▇█▇
val_loss,█▆▅▄▄▃▂▂▂▂▂▂▁▂▂▁▁▁▁▁
accuracy,0.75446
best_epoch,16
best_val_loss,0.47367
cohen_kappa,0.47739


In [ ]:
import tensorflow_addons as tfa
# load the best model
wandb.init(mode="disabled")
api = wandb.Api()
artifact = api.artifact('aueb/Deep_Learning_2/model-Stacked_frozen:v18', type='model')
artifact.download()

metrics = [tfa.metrics.CohenKappa(name="cohen_kappa", num_classes=2),
           'accuracy']

artifact_model = tf.keras.models.load_model("./artifacts/model-Stacked_frozen:v18", custom_objects={"cohen_kappa": metrics[0]})

print(artifact_model.summary())

wandb: Downloading large artifact model-Stacked_frozen:v18, 82.66MB. 5 files... 
wandb:   5 of 5 files downloaded.  
Done. 0:0:1.3


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 mobilenetv2_1.00_224 (Function  (None, 7, 7, 1280)  2257984     ['Input[0][0]']                  
 al)                                                                                              
                                                                                                  
 densenet169 (Functional)       (None, 7, 7, 1664)   12642880    ['Input[0][0]']                  
                                                                                              

In [ ]:
test_iterator.reset
_eval = artifact_model.evaluate(test_iterator)
_eval

32/32 [==============================] - 36s 1s/step - loss: 0.5120 - cohen_kappa: 0.4945 - accuracy: 0.7661


[0.5119682550430298, 0.49445825815200806, 0.7661391496658325]